In [ ]:
!pip install -U transformers datasets accelerate peft bitsandbytes trl==0.8.6 faiss-cpu

In [ ]:
!pip install -U langchain langchain-core langchain-community langchain_huggingface

In [ ]:
!pip install langchain_teddynote

#모델 불러오기

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'  # 사용할 모델의 ID를 지정합니다.'beomi/KoAlpaca-Polyglot-12.8B'#
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)  # 지정된 모델의 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(model_id)  # 지정된 모델을 로드합니다.

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#파이프라인

In [ ]:
pipe = pipeline("text-generation", model=model,
                tokenizer=tokenizer, max_new_tokens=512, eos_token_id=[128009, 128009]) #종료 토큰 설정
# HuggingFacePipeline 객체를 생성하고, 생성된 파이프라인을 전달합니다.
hf = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
/tmp/ipython-input-3453388933.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)


#conversationChain 이용

##ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

/tmp/ipython-input-3262330552.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
from langchain_core.prompts import PromptTemplate
template = '''
Conversation History:
{history}
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

In [ ]:
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
history = memory.load_memory_variables({})['history']
conversation = ConversationChain(
    llm = hf,
    memory=memory,
    prompt = prompt,
)
response = conversation.predict(history=history, input= "안녕")
print(response)

/tmp/ipython-input-2264897707.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(



Conversation History:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
안녕<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 무엇을 도와드릴까요?


In [ ]:
# 메모리에 저장된 대화 내용 확인
memory.load_memory_variables({})['history']

"Human: 안녕\nAI: \nConversation History:\n\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n안녕<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n안녕하세요! 무엇을 도와드릴까요?"

In [ ]:
response = conversation.predict(input= "우리가 지난번에 무슨 대화를 했었지?")
print(response) #전체 내용을 그대로 출력함


Conversation History:
Human: 안녕
AI: 
Conversation History:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
안녕<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 무엇을 도와드릴까요?
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
우리가 지난번에 무슨 대화를 했었지?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Conversation History:
Human: 안녕
AI: 
Conversation History:




#conversationChain 이용x

In [ ]:
from langchain.prompts import PromptTemplate

#프롬프트 및 체인 생성

template = '''

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요. 필요하다면 이전 대화를 참고하세요.<|eot_id|>
<|start_header_id|>이전 대화 기록<|end_header_id|>
{history}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''
prompt = PromptTemplate.from_template(template)

chain = prompt | hf


##요약x

In [ ]:
def chatbot(user_input, memory):
  history = memory.load_memory_variables({})['history']
  response=chain.invoke({"history": history, "user_input": user_input})
  print(f'\n-----------------------------------\n이전 대화 기록\n{history}\n-----------------------------------\n')
  response = response.split('<|start_header_id|>assistant<|end_header_id|>\n')[1]
  print(f"출력: {response}\n\n")

  #메모리에 직접 저장
  memory.save_context(
      inputs={"Human":user_input},
      outputs={"AI":response}
  )

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

while True:
  user_input = input("입력: ")
  if user_input == "종료":
      break
  chatbot(user_input, memory)

입력: 안녕 내 이름은 '테디'야

-----------------------------------
이전 대화 기록

-----------------------------------

출력: 안녕하세요! 테디라는 이름이 참 귀여워요. 무엇을 도와드릴까요?


입력: 파리 여행할 때 꼭 가야 할 곳 추천해줘

-----------------------------------
이전 대화 기록
Human: 안녕 내 이름은 '테디'야
AI: 안녕하세요! 테디라는 이름이 참 귀여워요. 무엇을 도와드릴까요?
-----------------------------------

출력: 파리 여행을 계획 중이시군요! 파리는 아름다운 건축물, 풍부한 문화, 그리고 맛있는 음식으로 유명한 도시입니다. 다음은 파리 여행 중 꼭 방문해야 할 곳들입니다:

1. **에펠탑 (Eiffel Tower)**:
   - 파리 최고의 상징이자 가장 인기가 있는 관광지입니다. 일몰 때는 아름다운 전망을 감상할 수 있습니다.

2. **루브르 박물관 (Louvre Museum)**:
   - 세계에서 가장 큰 박물관 중 하나로, 모나리자, 비너스상, 그로타르코와 같은 유명 작품들을 소장하고 있습니다.

3. **노트르담 대성당 (Notre-Dame Cathedral)**:
   - 고딕 건축의 대표적인 예로, 파리의 역사와 문화를 느낄 수 있는 곳입니다.

4. **샹젤리제 거리 (Champs-Élysées)**:
   - 파리의 중심부에 위치한 거리로서, 아름다운 빌딩과 박물관, 극장 등이 위치해 있습니다.

5. **라산젤리사 섬 (Île de la Cité)**:
   - 세느강에 위치한 섬으로, 노트르담 대성당, 파리 대법원, 생트 샤페르크랄 감옥 등이 있습니다.

6. **샤르르르 드 몽마르트 (Montmartre)**:
   - 예술가들이 많이 모여든 곳으로, 아름다운 교회와 좁은 거리, 그리고 예술 작품을 감상할 수 있는 곳입니다.

7. **세느강 크루즈 (Seine River Crui

In [ ]:
memory.load_memory_variables({})['history']

"Human: 안녕 내 이름은 '테디'야\nAI: 안녕하세요! 테디라는 이름이 참 귀여워요. 무엇을 도와드릴까요?\nHuman: 파리 여행할 때 꼭 가야 할 곳 추천해줘\nAI: 파리 여행을 계획 중이시군요! 파리는 아름다운 건축물, 풍부한 문화, 그리고 맛있는 음식으로 유명한 도시입니다. 다음은 파리 여행 중 꼭 방문해야 할 곳들입니다:\n\n1. **에펠탑 (Eiffel Tower)**:\n   - 파리 최고의 상징이자 가장 인기가 있는 관광지입니다. 일몰 때는 아름다운 전망을 감상할 수 있습니다.\n\n2. **루브르 박물관 (Louvre Museum)**:\n   - 세계에서 가장 큰 박물관 중 하나로, 모나리자, 비너스상, 그로타르코와 같은 유명 작품들을 소장하고 있습니다.\n\n3. **노트르담 대성당 (Notre-Dame Cathedral)**:\n   - 고딕 건축의 대표적인 예로, 파리의 역사와 문화를 느낄 수 있는 곳입니다.\n\n4. **샹젤리제 거리 (Champs-Élysées)**:\n   - 파리의 중심부에 위치한 거리로서, 아름다운 빌딩과 박물관, 극장 등이 위치해 있습니다.\n\n5. **라산젤리사 섬 (Île de la Cité)**:\n   - 세느강에 위치한 섬으로, 노트르담 대성당, 파리 대법원, 생트 샤페르크랄 감옥 등이 있습니다.\n\n6. **샤르르르 드 몽마르트 (Montmartre)**:\n   - 예술가들이 많이 모여든 곳으로, 아름다운 교회와 좁은 거리, 그리고 예술 작품을 감상할 수 있는 곳입니다.\n\n7. **세느강 크루즈 (Seine River Cruise)**:\n   - 파리의 아름다운 건축물들을 따라 흐르는 세느강을 따라 항해하며 파리를 관광할 수 있는 좋은 방법입니다.\n\n8. **라 도마니 파르크 (Le Jardin des Tuileries)**:\n   - 로열궁과 인접한 공원으로, 아름다운 정원과 미술관이 있습니다.\n\n9. **오르세 미술관 (Orsay Museum)**:\n   - 19

##conversationSummaryMemory

In [ ]:
!pip install langchain_google_genai

In [ ]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory =  (
    llm=llm,
    max_token_limit=50,
    return_messages=True
    )

/tmp/ipython-input-3373199450.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [ ]:
def chatbot(user_input, memory):
  history = memory.load_memory_variables({})['history']
  response=chain.invoke({"history": history, "user_input": user_input})
  print(f'\n-----------------------------------\n이전 대화 기록\n{history}\n-----------------------------------\n')
  response = response.split('<|start_header_id|>assistant<|end_header_id|>\n')[1]
  print(f"출력: {response}\n\n")

  memory.save_context(
      inputs={"human":user_input},
      outputs={"ai":response},
  )

In [ ]:
while True:
  user_input = input("입력: ")
  if user_input == "종료":
      break
  chatbot(user_input, memory)

입력: 안녕 내 이름은 '테디'야

-----------------------------------
이전 대화 기록
[]
-----------------------------------

출력: 안녕하세요 테디! 저는 AI 어시스턴트입니다. 무엇을 도와드릴까요?


입력: 파리 여행할 때 꼭 가야 할 곳 추천해줘

-----------------------------------
이전 대화 기록
[HumanMessage(content="안녕 내 이름은 '테디'야", additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 테디! 저는 AI 어시스턴트입니다. 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]
-----------------------------------

출력: 파리는 세계에서 가장 아름다운 도시 중 하나로, 역사, 미술, 식음료 등 다양한 매력을 자랑합니다. 파리 여행할 때 꼭 방문해야 할 곳을 추천해 드리겠습니다.

1. **에펠탑 (Eiffel Tower)**: 파리의 상징이자 세계적인 랜드마크. 태양이 지구를 돌고 있는 것처럼 보이는 일몰 때는 더욱 아름답습니다.

2. **루브르 박물관 (Louvre Museum)**: 세계에서 가장 큰 미술관 중 하나로, 모나리자, 뫼르 드 몽마르트르, 비너스상 등 수많은 명작들을 감상할 수 있습니다.

3. **노트르담 대성당 (Notre-Dame Cathedral)**: 중세 시대의 아름다운 건축물로, 영화 '인사이드 아웃'에서 등장한 장소로도 유명합니다.

4. **샤를 드 골 광장 (Place Charles de Gaulle)**: 트라이앵글 무대와 아르코의 아름다운 조화가 인상적인 광장입니다. 주변에는 많은 식당과 카페가 있어 휴식 공간으로 활용할 수 있습니다.

5. **몽마르트르 언덕 (Montmartre)**: 예술가들이 많이 활동했던 지역으로, 아름다운 교회와 화려

In [ ]:
memory.load_memory_variables({})['history']

[SystemMessage(content="Teddy asked an AI assistant for recommendations on must-see places in Paris and received eight suggestions: the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Place Charles de Gaulle, Montmartre, a Seine River cruise, the Musée d'Orsay, and La Rocher.  The AI also provided recommendations for Parisian food, including crêpes, steak tartare, a Champagne cocktail, courgette fritters, pâté, Jambon de Bayonne, macarons, arrabbiata, and a croque monsieur. The conversation then shifted to the AI answering questions in Korean. The human asked the AI what the capital of South Korea is (Seoul), and then asked if the AI remembered their name. The AI confirmed it remembered the human's name and its previous responses, including the Paris recommendations (both places and food),  offering further assistance.", additional_kwargs={}, response_metadata={})]

##직접 요약

In [ ]:
def chatbot(user_input, history):
  response=chain.invoke({"history": history, "user_input": user_input})
  response = response.split('<|start_header_id|>assistant<|end_header_id|>\n')[1]
  print(f"출력: {response}\n")
  summary = summary_chain.invoke({"history":history, "new_lines":response}).content
  print(f'\n-----------------------------------\n요약 메세지\n{summary}\n-----------------------------------\n\n')
  return summary

In [ ]:
#요약용 프롬프트 & 체인 생성
summary_template =  """
새로운 대화와 기존 요약 내용을 기반으로 대화 내용을 요약해주세요.
기존 요약:
{history}
새로운 대화:
{new_lines}
요약:
"""

summary_prompt = PromptTemplate(
    input_variables=["history"],
    template=summary_template
)

summary_chain = summary_prompt | llm


history = ''
while True:
  user_input = input("입력: ")
  if user_input == "종료":
      break
  history = chatbot(user_input, history)

입력: 안녕 내 이름은 '테디'야
출력: 안녕하세요, 테디! 저는 당신을 도와드릴 준비가 되었습니다. 어떤 도움이 필요하신가요?


-----------------------------------
요약 메세지
요약:  테디가 도움을 요청했고, 도와줄 준비가 되었다고 응답했다.  테디가 어떤 도움을 필요로 하는지는 아직 명시되지 않았다.
-----------------------------------


입력: 파리 여행할 때 꼭 가야 할 곳 추천해줘
출력: 파리는 아름다운 건축물, 풍부한 문화, 그리고 맛있는 음식으로 유명한 도시입니다. 파리에 가면 꼭 방문해야 할 곳은 다음과 같습니다:

1. **에펠탑(Eiffel Tower)**: 파리의 상징이자 세계적으로 유명한 랜드마크입니다. 전망대에서 파리의 전경을 감상할 수 있습니다.

2. **루브르 박물관(Louvre Museum)**: 세계에서 가장 큰 박물관 중 하나로, 모나리자, 몰리에르의 저녁 식사, 비너스 스탠스 등 유명한 예술 작품들을 보실 수 있습니다.

3. **노트르담 대성당(Cathedral of Notre-Dame de Paris)**: 고딕 건축의 걸작으로, 고딕 건축의 역사와 문화를 체험할 수 있는 곳입니다.

4. **샤를 드 골 광장(Place Charles de Gaulle)**: 프랑스의 대통령을 기념하는 광장으로, 파리의 중심에 위치해 있습니다.

5. **세느강 크루즈(Sèine River Cruise)**: 파리의 아름다운 건축물과 역사적인 장소를 배를 타고 둘러볼 수 있는 방법입니다.

6. **몽마르트르 언덕(Montmartre)**: 예술가들이 많이 활동했던 지역으로, 아름다운 거리와 거리 공연, 그리고 생르베르트 대성당을 볼 수 있습니다.

7. **샤르르 드 골 광장 근처의 카페 데 두올(Cafe des Deux Magots)**: 파리의 유명한 카페로, 파리의 문화와 예술의 중심지로 알려져 있습니다.

8. **장 드 마리 광장(Place des Vosges)**:

In [ ]:
history

'테디는 에펠탑과 루브르 박물관 방문, 에스까르고와 코크 오 뱅 시식, 그리고 피에르레 드 루시옹 등 유명 요리사 및 레스토랑 정보 획득을 목표로 파리 여행 계획을 세우고 있다.  새로운 대화에서는 이전 대화 내용에 대한 정보 접근이 불가능함을 알 수 있다.'

#RAG(이전 대화 기록들) 추가

In [ ]:
!pip install chromadb sentence-transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

#임베딩 모델 설정
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipython-input-235625837.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import chromadb

#벡터 저장소 생성
persist_directory = "chroma_db"
vectordb = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
#검색기 정의 (가장 유사한 한 개 찾아옴)
retriever = vectordb.as_retriever(search_kwargs={"k": 1})


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
#RAG 체인 생성
rag_chain = (
    {"history": retriever | format_docs, "user_input": RunnablePassthrough()}
    | prompt
    | hf
    | StrOutputParser()
)

#멀티턴 대화 실행
print("AI와 대화를 시작합니다. '종료'를 입력하면 끝납니다.")
while True:
    user_input = input("입력: ")
    if user_input.lower() == "종료":
        break

    #RAG 체인 호출 및 답변 생성
    response = rag_chain.invoke(user_input)
    response = response.split('<|start_header_id|>이전 대화 기록<|end_header_id|>')[1].split('<|eot_id|>\n<|start_header_id|>user<|end_header_id|>')
    print(f"\n -----------------------------\n이전 대화 기록{response[0]}\n-------------------------------\n")
    response = response[1].split('<|start_header_id|>assistant<|end_header_id|>\n')[1]
    print(f"출력: {response}\n\n")

    #벡터 저장소에 저장
    vectordb.add_texts([f"Human: {user_input}\nAI: {response}"])

AI와 대화를 시작합니다. '종료'를 입력하면 끝납니다.
입력: 안녕 내 이름은 '테디'야

 -----------------------------
이전 대화 기록

-------------------------------

출력: 안녕하세요 테디님! 어떻게 지내세요? 무엇을 도와드릴까요?


입력: 파리 여행할 때 꼭 가야 할 곳 추천해줘

 -----------------------------
이전 대화 기록
Human: 안녕 내 이름은 '테디'야
AI: 안녕하세요 테디님! 어떻게 지내세요? 무엇을 도와드릴까요?
-------------------------------

출력: 파리, 세계적으로 유명한 도시 중 하나입니다. 이곳에는 다양한 문화, 역사, 예술, 음식 등이 풍부하게 존재하며, 다양한 관광지가 많습니다. 파리 여행을 계획하고 계시다면 꼭 방문해야 할 곳은 다음과 같습니다:

1. **에펠탑(Eiffel Tower)**:
   - 파리에서 가장 유명한 랜드마크 중 하나로, 세계적으로 알려진 이 탑은 파리의 전경을 한눈에 내려다볼 수 있는 전망대가 있습니다.

2. **루브르 박물관(Louvre Museum)**:
   - 세계에서 가장 큰 박물관 중 하나로, 레오나르도 다 빈치의 "모나리자"와 비에르조의 "미스 비ュ르" 같은 많은 명작이 전시되어 있습니다.

3. **노트르담 대성당(Notre-Dame Cathedral)**:
   - 중세 시대의 아름다운 건축물로, 파리의 중요한 문화적 상징 중 하나입니다. 2019년 화재로 인해 일부가 파손되었지만, 복원 작업이 진행 중입니다.

4. **몽마르트르 언덕(Montmartre)**:
   - 예술가들이 모여든 곳으로, 작은 거리와 상점, 카페가 가득한 이곳은 파리의 아름다운 풍경을 즐길 수 있는 곳입니다.

5. **샤르르르 루시앙 거리(Champs-Élysées)**:
   - 파리의 중심부에 위치한 이 거리는 아름다운 조성과 역사적인 건축물로 유명합니다. Arc de Triomphe와 같은 

In [ ]:
all_data = vectordb._collection.get(include=["metadatas", "documents"])
print(all_data)

{'ids': ['4c3843cf-9552-4cf3-ad15-526913bbb97c', 'c7f977b0-da70-459d-b16d-fb9a149bdc67', 'ac1c07b1-60d7-4911-8d76-c61ec54027cd', '3eca2569-5114-4111-b89b-e53caa2444ea', '28ed8d70-d383-4c58-bd97-914da11a54d1', '2c136fc2-df17-49ea-af42-ef0de2c7fd08'], 'embeddings': None, 'documents': ["Human: 안녕 내 이름은 '테디'야\nAI: 안녕하세요 테디님! 어떻게 지내세요? 무엇을 도와드릴까요?", 'Human: 파리 여행할 때 꼭 가야 할 곳 추천해줘\nAI: 파리, 세계적으로 유명한 도시 중 하나입니다. 이곳에는 다양한 문화, 역사, 예술, 음식 등이 풍부하게 존재하며, 다양한 관광지가 많습니다. 파리 여행을 계획하고 계시다면 꼭 방문해야 할 곳은 다음과 같습니다:\n\n1. **에펠탑(Eiffel Tower)**:\n   - 파리에서 가장 유명한 랜드마크 중 하나로, 세계적으로 알려진 이 탑은 파리의 전경을 한눈에 내려다볼 수 있는 전망대가 있습니다.\n\n2. **루브르 박물관(Louvre Museum)**:\n   - 세계에서 가장 큰 박물관 중 하나로, 레오나르도 다 빈치의 "모나리자"와 비에르조의 "미스 비ュ르" 같은 많은 명작이 전시되어 있습니다.\n\n3. **노트르담 대성당(Notre-Dame Cathedral)**:\n   - 중세 시대의 아름다운 건축물로, 파리의 중요한 문화적 상징 중 하나입니다. 2019년 화재로 인해 일부가 파손되었지만, 복원 작업이 진행 중입니다.\n\n4. **몽마르트르 언덕(Montmartre)**:\n   - 예술가들이 모여든 곳으로, 작은 거리와 상점, 카페가 가득한 이곳은 파리의 아름다운 풍경을 즐길 수 있는 곳입니다.\n\n5. **샤르르르 루시앙 거리(Champs-Élysées)**:\n 